In [ ]:
# Import PyTorch

import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Import matplotlib

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
class FashionMNISTModelV1(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_units):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU()
        )

    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)


In [ ]:
# Getting a dataset
train_data = datasets.FashionMNIST(
    root="data", # where to download data
    train=True,
    download=True, # download yes/no?
    transform=torchvision.transforms.ToTensor(), # transform on the data
    target_transform=None # how we want to transform the label
)

test_data = datasets.FashionMNIST(
    root="data", # where to download data
    train=False,
    download=True, # download yes/no?
    transform=torchvision.transforms.ToTensor(), # transform on the data
    target_transform=None # how we want to transform the label
)

100%|██████████| 26421880/26421880 [00:00<00:00, 115511510.85it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5629599.03it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 69730853.93it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 18841428.44it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
class_names = train_data.classes

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [ ]:
model_1 = FashionMNISTModelV1(input_shape=784,
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)

In [ ]:
# setup loss
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.1)

In [ ]:
from timeit import default_timer as timer
def print_train_time(start, end, device):
  total_time = end - start
  print(f"Train time on {device}: {total_time:.3f} seconds")


In [ ]:
torch.manual_seed(42)


def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device=device):
  """ Performs a training with model trying to learn on the dataloader"""
  train_loss, train_acc = 0, 0

# Set the seed and start the timer
  model.train()

  # Add a loop to iterate through the dataloader
  for batch, (X, y) in enumerate(train_dataloader):
    # Put data on target device
    X, y = X.to(device), y.to(device)

    # 1. Forward Pass
    y_pred = model(X)

    # 2. Calculate loss
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)}")

  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")

In [ ]:
def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device=device):
  """
  Performs a test step with the model given.
  """
  # Testing
  test_loss, test_acc = 0, 0
  model_1.eval()
  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      X_test, y_test = X_test.to(device), y_test.to(device)
      # 1. Forward Pass
      test_pred = model_1(X_test)
      # 2. Calculate Loss
      test_loss += loss_fn(test_pred, y_test)
      # 3. Calculate Accuracy
      test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f"\nTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

In [ ]:
# Import TQDM for progress bar
from tqdm.auto import tqdm
# Import timer from timeit
from timeit import default_timer as timer

# Start timer
train_time_start_on_gpu = timer()

# Set epochs
epochs = 3

# Create an optimization and evaluation loop using train and test step
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n--------")

  # Start training
  train_step(
      model=model_1,
      data_loader=train_dataloader,
      loss_fn=loss_fn,
      optimizer=optimizer,
      accuracy_fn=accuracy_fn,
      device=device)

  # Start testing
  test_step(
      model=model_1,
      data_loader=train_dataloader,
      loss_fn=loss_fn,
      accuracy_fn=accuracy_fn,
      device=device)

# End timer
train_time_end_on_gpu = timer()

# Calculate time it took to run.
total_train_time = print_train_time(train_time_start_on_gpu, train_time_end_on_gpu, device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
--------
Looked at 0/60000
Looked at 12800/60000
Looked at 25600/60000
Looked at 38400/60000
Looked at 51200/60000
Train loss: 1.04729 | Train acc: 64.61%

Test Loss: 0.9217 | Test Acc: 68.3706
Epoch: 1
--------
Looked at 0/60000
Looked at 12800/60000
Looked at 25600/60000
Looked at 38400/60000
Looked at 51200/60000
Train loss: 0.89166 | Train acc: 68.74%

Test Loss: 0.9226 | Test Acc: 68.1709
Epoch: 2
--------
Looked at 0/60000
Looked at 12800/60000
Looked at 25600/60000
Looked at 38400/60000
Looked at 51200/60000
Train loss: 0.72108 | Train acc: 74.20%

Test Loss: 0.7223 | Test Acc: 73.9816
Train time on cuda: 37.836 seconds
